<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## `pandas` Long Format, Wide Format, Pivot Tables, and Melting
_Instructor: Aymeric Flaisler_
___
<br>

This lesson is all about **transforming data** using `pandas`. Data transformation is the reorganization of your data set's rows and columns into a different, potentially **more useful shape and format**. 

The benefits of transforming your data include **better access to relevant information** and **streamlined data manipulation**. As you become more familiar with data sets and their associated operations, you will develop an intuition and appreciation for when it's better to **work row-wise or column-wise**.

Different data formats are better for different tasks. It takes time and experience to learn the distinctions. But, for now, we'll introduce the **common structures, transformations, and how to apply these transformations**.

### Learning Objectives
- Understand the differences between **long and wide format data**.
- Understand **pivot tables**.
- Practice transforming data between **long and wide** formats.
- Practice creating pivot tables.
- Learn how to avoid **common pitfalls and obstacles** in data transformation with `pandas`.


### Lesson Guide

- [Wide Format Data](#wide_format)
- [Load and Examine the NPAS Data](#load_nerdy)
- [Long Format Data](#long_format)
- [Using `pandas`' `.pivot_table()` Function: Long to Wide Format](#pivot_tables)
- [MultiIndex/Hierarchical Indices in `pandas`](#multiindex)
- [Using `pandas`' `.melt()` Function: Wide to Long Format](#melt)
- [Summarizing Data With `.pivot_table()` and Aggregate  Functions](#pivot_table_summarizing)
- [The Inner Workings of the MultiIndex](#examining_multiindex)
- [Getting Rid of the MultiIndex: "Flattening" Data](#multiindex_to_flat)
- [A Preface: Merging and Joining With Long and Wide Format Data](#merging_joining_preface)
- [`pandas`' `.merge()` function: Joining Long Format vs. Wide Format Data](#pandas_merge)


In [58]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<a id='wide_format'></a>

### Wide Format Data

---

Between "wide" and "long," **wide format data is the more intuitive**. It's also a common format for `.csv` files. You've already viewed multiple data sets in wide format throughout this course.

Wide format data is structured so that:

- Unique IDs, subjects, observations, etc. are represented as **rows**.
- Distinct information categories (**variables**) are represented as columns. In other words, there is a **column for every "variable"** with its own unique values.
- This format can often be a more compact matrix, particularly if little or no information is missing.
- It is **not as useful for SQL-style operations**: It can make it much harder or even impossible to **join tables together on a value**.
- It can be useful in `pandas` when you need to perform operations on variables **across columns**; for example, multiplying columns together to create a new column.
- It is the data format required for statistical modeling (with few exceptions).

<a id='load_nerdy'></a>

### Load and Examine the "Nerdy Personality Attributes" Data Set

---

This is a pre-cleaned and modified version of the full "Nerdy Personality Attributes" survey, which asked subjects to rate themselves based on questions related to "nerdiness" as well as more general personality traits such as openness and extraversion. Researches also collected demographic information from the subjects.

You can find the raw data [here](http://personality-testing.info/_rawdata/), along with many other sociological surveys.

In this modified version, for the sake of our example, some of the subjects provided data for the survey but not the demographic variables. Because there are missing values and the data is "messy," we have a data cleaning problem.

**Load the data (which is in wide format).** 

In [59]:
nerdy_wide_f = './datasets/NPAS_parsed_trunc_wide_missing.csv'

# load data and print the dimensions
nerdy_wide = pd.read_csv(nerdy_wide_f)

This data set is in a familiar format in which each column is a variable and each row contains an observation for that variable, corresponding to a distinct subject.

*Wide format implies that all of the information for one distinct subject **will be represented in the columns corresponding to that row**. A single subject should not be represented in multiple rows of data.*

In [60]:
# First let's print the header:
nerdy_wide.head()

,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,diagnosed_autistic,disorganized,education,engnat,enjoy_learning,excited_about_research,extraverted,familysize,gender,hand,hobbies_over_people,in_advanced_classes,intelligence_over_appearance,interested_science,introspective,libraries_over_publicspace,like_dry_topics,like_hard_material,like_science_fiction,like_superheroes,major,married,online_over_inperson,opennness,play_many_videogames,playes_rpgs,prefer_fictional_people,race_arab,race_asian,race_black,race_hispanic,race_native_american,race_native_austrailian,race_nerdy,race_white,read_tech_reports,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,7.0,2.0,1.0,NaN,NaN,5.0,5.0,1.0,NaN,NaN,NaN,4.0,5.0,3.0,3.0,5.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,4.0,6.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,2.0,5.0,4.0,1.0,3.0,4.0,5.0,3.0,2.0,1.0,1.0,4.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,4.0,biophysics,1.0,3.0,5.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,2.0,6.0,3.0,2.0,5.0,5.0,1.0,2.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,biology,1.0,5.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
3,3,5.0,NaN,4.0,4.0,5.0,7.0,5.0,1.0,2.0,7.0,2.0,1.0,NaN,NaN,1.0,5.0,7.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,2.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,5.0,7.0,5.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,NaN,2.0,5.0,5.0,6.0,NaN,NaN,5.0,5.0,4.0
4,4,4.0,NaN,3.0,5.0,5.0,6.0,4.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN,4.0,4.0,2.0,NaN,NaN,NaN,4.0,4.0,4.0,4.0,1.0,4.0,4.0,5.0,4.0,4.0,NaN,NaN,5.0,7.0,3.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,6.0,0.0,5.0,5.0,NaN,NaN,5.0,4.0,1.0


**Check to see how many null values there are per column.**

*Tips:* An easy way is to use the `.isnull()` method associated with the `.sum()`

In [61]:
# Now let's count the null values by column:

nerdy_wide.isnull().sum()
#nerdy_wide.shape

subject_id                        0
academic_over_social              0
age                             691
anxious                           0
bookish                           0
books_over_parties                0
calm                              0
collect_books                     0
conventional                      0
critical                          0
dependable                        0
diagnosed_autistic                0
disorganized                      0
education                       691
engnat                          691
enjoy_learning                    0
excited_about_research            0
extraverted                       0
familysize                      691
gender                          691
hand                            691
hobbies_over_people               0
in_advanced_classes               0
intelligence_over_appearance      0
interested_science                0
introspective                     0
libraries_over_publicspace        0
like_dry_topics             

The 691 missing demographic variables are intentional (I specifically enforced that only 700 of the subjects would have demographic information).

However, we can see that the `major` variable has 970 missing values. This was not an intentional change.

At this point, if we were to just **drop all the rows that have any null values, we would lose at least 970 rows** because of the missing `major` variable.

With a numeric column, this would be hard to avoid without "imputing" some number to fill in those values. In the simplest case, **imputing the mean or median for missing numeric values** is a common fix (but not ideal).

With a **categorical variable** like `major`, we have the luxury of replacing the missing values with a new category label that stands for "missing." 

**Replace the missing `major` column values with `unknown`.**

In [62]:
# first create a mask for the missing values in the major column:
nerdy_wide['major'].fillna('missing',inplace=True)

# set missing values in major to "unknown":

In [63]:
# if all goes right you should not have any missing values left
print (nerdy_wide.major.isnull().sum())

0


<a id='long_format'></a>

### Long Format Data

---

Now, we can load the same data — this time in the format commonly called "long."

Long format data is structured so that:

- There are potentially multiple `ID` (identification) columns.
- There are pairs of columns such as `variable:value` that match a variable key to a value (In the simplest case, there would be a single `variable` column and a single `value` column).
- The `variable` column corresponds to the multiple variable columns in a wide format data set. Instead of a column for each variable, you have a row for each `variable:value` pair *per ID*. 
- This is a standard format for SQL databases because it makes it easier to join different tables together with keys.

**Load the long format of the same data below.**

In [64]:
nerdy_long_f = './datasets/NPAS_parsed_trunc_long_missing.csv'

# load long data and print the dimensions
nerdy_long = pd.read_csv(nerdy_long_f)

You can see that the long format data has far more rows than the wide data set but only three columns.

Below you can view the three columns: `subject_id`, `variable`, and `value`.

**`subject_id:`**
- This is the primary "key" or `ID` column. Each `subject_id` will have corresponding entries in the `variable` column — one for each row.

**`variable:`**
- This column indicates the variable with which the item in the `value` column corresponds.

**`value:`**

- This contains all values for all variables for all IDs. Essentially, every cell in the wide data set except the `subject_id` is listed in this column.

In [65]:
# print the header:
nerdy_long.head()

,subject_id,variable,value
0,1,education,4.0
1,2,education,3.0
2,5,education,2.0
3,6,education,2.0
4,7,education,2.0


**Print out the unique values in the `variable` column.**

You can see that the unique values in the `variable` column correspond to the column headers in the wide format data.

*Tips: use the .unique() method*

In [66]:
# print the unique values in the variable column:
nerdy_long.variable.unique()

array(['education', 'urban', 'gender', 'engnat', 'age', 'hand',
       'religion', 'voted', 'married', 'familysize', 'major',
       'race_white', 'race_nerdy', 'race_native_american',
       'writing_novel', 'read_tech_reports', 'online_over_inperson',
       'introspective', 'hobbies_over_people', 'books_over_parties',
       'bookish', 'libraries_over_publicspace', 'race_native_austrailian',
       'like_hard_material', 'race_hispanic', 'diagnosed_autistic',
       'play_many_videogames', 'race_arab', 'race_asian',
       'interested_science', 'playes_rpgs', 'in_advanced_classes',
       'collect_books', 'intelligence_over_appearance',
       'watch_science_shows', 'academic_over_social',
       'like_science_fiction', 'like_dry_topics', 'race_black', 'calm',
       'disorganized', 'extraverted', 'dependable', 'critical',
       'opennness', 'anxious', 'sympathetic', 'reserved', 'conventional',
       'was_odd_child', 'prefer_fictional_people', 'enjoy_learning',
       'excited_abou

In [67]:
# count the unique subject ids:
nerdy_long.subject_id.nunique()

1391

**Replace the missing values in `major` with `unknown` in the long format data set.**

The process for replacing data will be different because of the format. Using logical selection masks with `pandas`' `.loc` syntax is the preferable way to do this.

In [68]:
# Identify the missing values in major:
sum(nerdy_long.value.isnull())

279

In [69]:
# replace the missing values for major in the long dataset with "unknown":
nerdy_mask=(nerdy_long.variable == 'major') & (nerdy_long.value.isnull())
nerdy_long.loc(major_mask,'value')='unknown'

SyntaxError: can't assign to function call (<ipython-input-69-87cf34e54a5e>, line 3)

In [ ]:
# check that there is no missing values left:
print(nerdy_long[nerdy_long.variable == 'major'].isnull().sum())

# you should get only 0s

<a id='pivot_tables'></a>

### `Pandas`' `.pivot_table()` Function: Long to Wide Format

---

The `pd.pivot_table()` function is a powerful tool for both transforming data from long to wide format as well as summarizing data with user-supplied functions.

First, we'll look at transforming the long format data back into the wide format using the `.pivot_table()` function.

**Important parameters for the `.pivot_table()` function include:**

> The `pivot_table()` function takes a DataFrame to pivot as its first argument. 
    
- **`columns`**: This is the list of columns in the long format data to be transformed back into columns in the wide format. After pivoting, each unique value in the long format column becomes a header in the wide format.
- **`values`**: A single column indicating the values to use when pivoting and filling the new wide format columns.
- **`index`**: Columns in the long format data that are index variables. These will be left as single columns, not spread out by unique value like in the `columns` parameter.
- **`aggfunc`**: Often `.pivot_table()` is used to perform a summary of the data. `aggfunc` stands for "aggregation function." It's required and defaults to `np.mean()`. You can also insert your own function, which we'll demonstrate below.
- **`fill_value`**: If a cell is missing for the wide format data, this value will fill it in.
    
Next we'll put in our own function — `select_item_or_nan()` — to the `aggfunc` keyword argument. Because my `subject_id` column has a single variable value for each ID, I just want the single element in the long format value cell. My data is messy, so I have to write a function to check for places it could break. 

**Note:** Passed into my function, `x` will be a Series object. I pull out the first element of that using the `.iloc` indexer.

### Let's make sure value has only values:

*Note: The lambda operator or lambda function is used for creating small, one-time and anonymous function objects in Python. This is not the object of this lesson. We will cover it at a later stage. Do not worry about understanding it for now.*

In [70]:
def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [71]:
# mask with true or false if we can convert to a numerical value
mask = nerdy_long.value.map(lambda x: is_float(x))

#### Now remove non numeric values using the mask

In [72]:
nerdy_long_only_num = nerdy_long[mask].copy()
nerdy_long_only_num.reset_index(drop=True, inplace=True)

In [73]:
nerdy_long_only_num.shape

(69874, 3)

#### Convert the column `value`  from the dataframe `nerdy_long_only_num` to float

In [77]:
# A:
nerdy_long_only_num['value']=nerdy_long_only_num['value'].map(lambda x: float(x) if is_float(x) else x)
nerdy_long_only_num.head()

,subject_id,variable,value
0,1,education,4.0
1,2,education,3.0
2,5,education,2.0
3,6,education,2.0
4,7,education,2.0


#### Finally pivot the data on subject_id and variable using .pivot()

In [79]:
nerdy_long_only_num.pivot(index='subject_id',columns='variable').head()

value                                                \
variable   academic_over_social   age anxious bookish books_over_parties calm   
subject_id                                                                      
0                           5.0   NaN     1.0     5.0                5.0  7.0   
1                           2.0  50.0     4.0     4.0                4.0  6.0   
2                           5.0  22.0     7.0     5.0                5.0  2.0   
3                           5.0   NaN     4.0     4.0                5.0  7.0   
4                           4.0   NaN     3.0     5.0                5.0  6.0   

                                                                              \
variable   collect_books conventional critical dependable diagnosed_autistic   
subject_id                                                                     
0                    5.0          1.0      1.0        7.0                2.0   
1                    5.0          1.0      3.0        5.0                2.0   
2                    5.0          1.0      6.0        3.0                2.0   
3                    5.0          1.0      2.0        7.0                2.0   
4                    4.0          2.0      5.0        4.0                2.0   

                                                         \
variable   disorganized education engnat enjoy_learning   
subject_id                                                
0                   1.0       NaN    NaN            5.0   
1                   5.0       4.0    1.0            3.0   
2                   6.0       3.0    2.0            5.0   
3                   1.0       NaN    NaN            1.0   
4                   5.0       NaN    NaN            4.0   

                                                                      \
variable   excited_about_research extraverted familysize gender hand   
subject_id                                                             
0                             5.0         1.0        NaN    NaN  NaN   
1                             4.0         5.0        3.0    2.0  1.0   
2                             5.0         1.0        2.0    2.0  1.0   
3                             5.0         7.0        NaN    NaN  NaN   
4                             4.0         2.0        NaN    NaN  NaN   

                                                    \
variable   hobbies_over_people in_advanced_classes   
subject_id                                           
0                          4.0                 5.0   
1                          1.0                 4.0   
2                          5.0                 5.0   
3                          5.0                 5.0   
4                          4.0                 4.0   

                                                                          \
variable   intelligence_over_appearance interested_science introspective   
subject_id                                                                 
0                                   3.0                3.0           5.0   
1                                   3.0                4.0           3.0   
2                                   5.0                5.0           5.0   
3                                   5.0                5.0           2.0   
4                                   4.0                4.0           1.0   

                                                                          \
variable   libraries_over_publicspace like_dry_topics like_hard_material   
subject_id                                                                 
0                                 5.0             3.0                5.0   
1                                 5.0             1.0                3.0   
2                                 4.0             5.0                4.0   
3                                 5.0             3.0                5.0   
4                                 4.0             4.0                5.0   

                                   

### Now with all the data and .pivot_table()

In [80]:
# First we define a custom function that returns a value if it exist and NaN if not
def select_item_or_nan(x):
    x = x.iloc[0]
#     print(x, type(x))
    if len(x) == 0:
        return np.nan
    else:
        return x
    

In [81]:
# This will take a few seconds to run.
nerdy_wide_pv = pd.pivot_table(nerdy_long, columns=['variable'], values='value',
                            index=['subject_id'], aggfunc=select_item_or_nan , fill_value=np.nan)
# 'pv' for 'pivot version.'
nerdy_wide_pv.head()

variable,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,diagnosed_autistic,disorganized,education,engnat,enjoy_learning,excited_about_research,extraverted,familysize,gender,hand,hobbies_over_people,in_advanced_classes,intelligence_over_appearance,interested_science,introspective,libraries_over_publicspace,like_dry_topics,like_hard_material,like_science_fiction,like_superheroes,major,married,online_over_inperson,opennness,play_many_videogames,playes_rpgs,prefer_fictional_people,race_arab,race_asian,race_black,race_hispanic,race_native_american,race_native_austrailian,race_nerdy,race_white,read_tech_reports,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,7.0,2.0,1.0,NaN,NaN,5.0,5.0,1.0,NaN,NaN,NaN,4.0,5.0,3.0,3.0,5.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,4.0,6.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,2.0,5.0,4.0,1.0,3.0,4.0,5.0,3.0,2.0,1.0,1.0,4.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,4.0,biophysics,1.0,3.0,5.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,2.0,6.0,3.0,2.0,5.0,5.0,1.0,2.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,biology,1.0,5.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
3,5.0,NaN,4.0,4.0,5.0,7.0,5.0,1.0,2.0,7.0,2.0,1.0,NaN,NaN,1.0,5.0,7.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,2.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,5.0,7.0,5.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,NaN,2.0,5.0,5.0,6.0,NaN,NaN,5.0,5.0,4.0
4,4.0,NaN,3.0,5.0,5.0,6.0,4.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN,4.0,4.0,2.0,NaN,NaN,NaN,4.0,4.0,4.0,4.0,1.0,4.0,4.0,5.0,4.0,4.0,NaN,NaN,5.0,7.0,3.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,6.0,0.0,5.0,5.0,NaN,NaN,5.0,4.0,1.0


<a id='multiindex'></a>

### MultiIndex/Hierarchical Indices in `pandas`

---

First, let's reload a fresh copy of the data:

In [82]:
# let's reload the data
nerdy_long = pd.read_csv('./datasets/nerdy_long.csv')

In [83]:
def select_item_or_nan(x):
#     print(type(x))
    x = x.iloc[0]
    if len(x) == 0:
        return np.nan
    else:
        return x

In [84]:
# This will take a few seconds to run.
nerdy_wide_pv = pd.pivot_table(nerdy_long, columns=['variable'], values='value',
                            index=['subject_id'], aggfunc=select_item_or_nan , fill_value=np.nan)


In the header, you can see that the format of the new wide data **is *not* the same as our originally loaded wide format**. `pandas` implements something called **MultiIndexing** or **hierarchical indexing**, which allows for "tiered" row and column labels.

Right now the MultiIndexing is not terrible but can get **confusing and annoying**, which we will experience later in this lesson.

The main difference is that we now have a `variable` name in the top left corner, which is **"labeling"** our columns (and corresponds to the name of our original column in the long format data). The row indexer has become our **single key/ID variable**, `subject_id`. The columns are what we would expect here: **Each one is a variable**, like in the original wide format data.

In [85]:
# print the header of the widened dataset
nerdy_wide_pv.head()

variable,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,diagnosed_autistic,disorganized,education,engnat,enjoy_learning,excited_about_research,extraverted,familysize,gender,hand,hobbies_over_people,in_advanced_classes,intelligence_over_appearance,interested_science,introspective,libraries_over_publicspace,like_dry_topics,like_hard_material,like_science_fiction,like_superheroes,major,married,online_over_inperson,opennness,play_many_videogames,playes_rpgs,prefer_fictional_people,race_arab,race_asian,race_black,race_hispanic,race_native_american,race_native_austrailian,race_nerdy,race_white,read_tech_reports,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,7.0,2.0,1.0,NaN,NaN,5.0,5.0,1.0,NaN,NaN,NaN,4.0,5.0,3.0,3.0,5.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,4.0,6.0,5.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,2.0,5.0,4.0,1.0,3.0,4.0,5.0,3.0,2.0,1.0,1.0,4.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,4.0,biophysics,1.0,3.0,5.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,2.0,6.0,3.0,2.0,5.0,5.0,1.0,2.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,biology,1.0,5.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
3,5.0,NaN,4.0,4.0,5.0,7.0,5.0,1.0,2.0,7.0,2.0,1.0,NaN,NaN,1.0,5.0,7.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,2.0,5.0,3.0,5.0,5.0,5.0,NaN,NaN,5.0,7.0,5.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,NaN,2.0,5.0,5.0,6.0,NaN,NaN,5.0,5.0,4.0
4,4.0,NaN,3.0,5.0,5.0,6.0,4.0,2.0,5.0,4.0,2.0,5.0,NaN,NaN,4.0,4.0,2.0,NaN,NaN,NaN,4.0,4.0,4.0,4.0,1.0,4.0,4.0,5.0,4.0,4.0,NaN,NaN,5.0,7.0,3.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,NaN,6.0,0.0,5.0,5.0,NaN,NaN,5.0,4.0,1.0


**Drop the null values from our recreated wide format data. How many unique subjects do we have?**

Remember our `subject_id` is now the **index**, so we can access it using the `.index` attribute.

In [86]:
# drop the null values and count unique subjects
nerdy_wide_pv.dropna(inplace=True)

**Convert the `subject_id` index back into a column (ie: reset the index without dropping it)**

We can use the DataFrame function `.reset_index()` to move `subject_id` into a column and create a new index. We now have a DataFrame with the same format we loaded the original wide format data in previously. The only exception is that we still have the `variable` column label.

In [90]:
# convert the index to a column
nerdy_wide_pv.reset_index(inplace=True)
nerdy_wide_pv.head()

variable,level_0,index,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,diagnosed_autistic,disorganized,education,engnat,enjoy_learning,excited_about_research,extraverted,familysize,gender,hand,hobbies_over_people,in_advanced_classes,intelligence_over_appearance,interested_science,introspective,libraries_over_publicspace,like_dry_topics,like_hard_material,like_science_fiction,like_superheroes,major,married,online_over_inperson,opennness,play_many_videogames,playes_rpgs,prefer_fictional_people,race_arab,race_asian,race_black,race_hispanic,race_native_american,race_native_austrailian,race_nerdy,race_white,read_tech_reports,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,0,0,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,2.0,5.0,4.0,1.0,3.0,4.0,5.0,3.0,2.0,1.0,1.0,4.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,4.0,biophysics,1.0,3.0,5.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
1,1,1,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,2.0,6.0,3.0,2.0,5.0,5.0,1.0,2.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,biology,1.0,5.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
2,2,2,5,4.0,18.0,5.0,3.0,4.0,4.0,4.0,4.0,3.0,5.0,2.0,3.0,2.0,1.0,2.0,5.0,2.0,3.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,Geology,1.0,3.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,5.0,4.0,5.0,4.0,3.0,2.0,4.0,5.0,3.0
3,3,3,6,4.0,18.0,1.0,4.0,5.0,6.0,5.0,1.0,1.0,2.0,2.0,5.0,2.0,1.0,1.0,4.0,3.0,2.0,1.0,2.0,3.0,3.0,3.0,5.0,1.0,3.0,5.0,1.0,5.0,3.0,missing,1.0,3.0,7.0,3.0,4.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,5.0,5.0,5.0,5.0,2.0,2.0,1.0,4.0,1.0
4,4,4,7,3.0,21.0,7.0,3.0,5.0,1.0,5.0,4.0,6.0,5.0,2.0,7.0,2.0,1.0,4.0,3.0,3.0,3.0,2.0,1.0,4.0,3.0,5.0,5.0,5.0,4.0,5.0,4.0,3.0,2.0,missing,1.0,4.0,6.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,12.0,5.0,5.0,5.0,6.0,2.0,1.0,3.0,3.0,3.0


**Remove the column label.**

You can remove the column label (which can be confusing during print statements) by setting the `.columns.name` attribute to `None`.

In [99]:
# remove the columns label
nerdy_wide_pv.columns.name=None
nerdy_wide_pv.head()

,level_0,index,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,diagnosed_autistic,disorganized,education,engnat,enjoy_learning,excited_about_research,extraverted,familysize,gender,hand,hobbies_over_people,in_advanced_classes,intelligence_over_appearance,interested_science,introspective,libraries_over_publicspace,like_dry_topics,like_hard_material,like_science_fiction,like_superheroes,major,married,online_over_inperson,opennness,play_many_videogames,playes_rpgs,prefer_fictional_people,race_arab,race_asian,race_black,race_hispanic,race_native_american,race_native_austrailian,race_nerdy,race_white,read_tech_reports,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,0,0,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,2.0,5.0,4.0,1.0,3.0,4.0,5.0,3.0,2.0,1.0,1.0,4.0,3.0,4.0,3.0,5.0,1.0,3.0,4.0,4.0,biophysics,1.0,3.0,5.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
1,1,1,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,2.0,6.0,3.0,2.0,5.0,5.0,1.0,2.0,2.0,1.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,3.0,biology,1.0,5.0,6.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
2,2,2,5,4.0,18.0,5.0,3.0,4.0,4.0,4.0,4.0,3.0,5.0,2.0,3.0,2.0,1.0,2.0,5.0,2.0,3.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,Geology,1.0,3.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,5.0,4.0,5.0,4.0,3.0,2.0,4.0,5.0,3.0
3,3,3,6,4.0,18.0,1.0,4.0,5.0,6.0,5.0,1.0,1.0,2.0,2.0,5.0,2.0,1.0,1.0,4.0,3.0,2.0,1.0,2.0,3.0,3.0,3.0,5.0,1.0,3.0,5.0,1.0,5.0,3.0,missing,1.0,3.0,7.0,3.0,4.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,5.0,5.0,5.0,5.0,2.0,2.0,1.0,4.0,1.0
4,4,4,7,3.0,21.0,7.0,3.0,5.0,1.0,5.0,4.0,6.0,5.0,2.0,7.0,2.0,1.0,4.0,3.0,3.0,3.0,2.0,1.0,4.0,3.0,5.0,5.0,5.0,4.0,5.0,4.0,3.0,2.0,missing,1.0,4.0,6.0,3.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,12.0,5.0,5.0,5.0,6.0,2.0,1.0,3.0,3.0,3.0


<a id='melt'></a>

### Using pandas' `.melt()` Function: Wide to Long Format

---
First, let's reload a fresh copy of the data:

In [93]:
nerdy_wide_flat = pd.read_csv('./datasets/nerdy_wide_flat.csv')

**`.melt()`** is a function that essentially performs the inverse of `.pivot_table()` on DataFrames.

`.melt()` takes a DataFrame as its first argument. Additional arguments typically used with this function are:

- **`id_vars`**: The column or columns that will be ID variables. ID variables contain data points specified by the `variable` and `value` columns.
- **`value_vars`**: A list that specifies which columns should be converted into single `value` and `variable` columns.
- **`var_name`**: The header name of the `variable` column (default='variable').
- **`value_name`**: The header name of the `value` column (default='value').

**First, subset the wide format data into just columns: `['subject_id','anxious','booking','calm','major']`.**

In [103]:
# subset the wide data:

nerdy_subset = nerdy_wide_flat[['subject_id','anxious','bookish','calm','major']]
nerdy_subset.head()

,subject_id,anxious,bookish,calm,major
0,1,4.0,4.0,6.0,biophysics
1,2,7.0,5.0,2.0,biology
2,5,5.0,3.0,4.0,Geology
3,6,1.0,4.0,6.0,missing
4,7,7.0,3.0,1.0,missing


**Use `.melt()` on the subset with `id_vars=['subject_id','major']`.**

Print out the shape of the data and the header. The non-ID columns and their values are now represented by the `variable:value` column pair.

**Note**: When you only specify the `id_vars`, the remaining columns become part of the `variable` and `value` columns.

In [105]:
nerdy_sub_long = pd.melt(nerdy_subset, id_vars=['subject_id','major'])
nerdy_sub_long[nerdy_sub_long['subject_id']== 1]

,subject_id,major,variable,value
0,1,biophysics,anxious,4.0
700,1,biophysics,bookish,4.0
1400,1,biophysics,calm,6.0


If we don't specify `major` as an `id_var`, it will end up in the `variable` column.

In [107]:
### with two value_vars
nerdy_sub_long = pd.melt(nerdy_subset, id_vars='subject_id')
print (nerdy_subset.shape, nerdy_sub_long.shape)
nerdy_sub_long.head(4)

(700, 5) (2800, 3)


,subject_id,variable,value
0,1,anxious,4
1,2,anxious,7
2,5,anxious,5
3,6,anxious,1


In [109]:
### with all value_vars
nerdy_sub_long = pd.melt(nerdy_wide_flat, id_vars=['subject_id','major'], 
                         value_vars=['anxious','bookish','calm'])
print (nerdy_wide_flat.shape, nerdy_sub_long.shape)
nerdy_sub_long.head(4)

(700, 57) (2100, 4)


,subject_id,major,variable,value
0,1,biophysics,anxious,4.0
1,2,biology,anxious,7.0
2,5,Geology,anxious,5.0
3,6,missing,anxious,1.0


The more `id_vars` that we specify, the flatter our DataFrame will be. 

You can achieve the same result without having to subset the DataFrame first by simply specifying the `value_vars` keyword argument. The output DataFrame will then only contain the data specified in the `id_vars` and `value_vars` arguments.

**Create the same DataFrame with `.melt()` on the full wide data set, but select the columns to use with the `value_vars` argument.**

In [110]:
nerdy_sub_long = pd.melt(nerdy_wide_flat, id_vars=['subject_id','major'], 
                         value_vars=['anxious','bookish','calm'])

In [113]:
# print the datatypes
nerdy_sub_long.dtype()

AttributeError: 'DataFrame' object has no attribute 'dtype'

The `value` column is still a string, so we can convert it to a float.

In [ ]:
# ensure the value is a float

<a id='pivot_table_summarizing'></a>

### Summarizing Your Data With  `.pivot_table()` and Aggregate Functions

---
First, let's reload a fresh copy of the data:

In [114]:
nerdy_sub_long = pd.read_csv('./datasets/nerdy_sub_long.csv')

For those of you who have experience with Excel, `pandas`' `.pivot_table()` accomplishes the same thing. It's more powerful but harder to use than the spreadsheet version.

`.pivot_table()` can take in a variable, value, and index to group by and apply aggregate functions to summarize the data. 

**Note**: Be careful that your index variable is not pulling out unique rows (For example, `subject_id` by variable would only have one value to send into the aggregate functions).

Below, I am calling the `.pivot_table()` function with:

- The long format data as the first argument.
- `variable` specified as the columns that indicate the variable names (groups).
- `value` specified as the column that contains the data per variable.
- `major` as the index; the rows will be grouped by `major`.
- `np.mean`, `np.median`, `np.std`, and `len` as aggregate functions. These will be calculated for each `major-by-variable` group.
- A `fill_value` of `np.nan` for cells in the output table that have no data.

In [122]:
nerdy_major_summary = pd.pivot_table(nerdy_sub_long, columns=['variable'], values='value',
                                     index=['major'], aggfunc=[np.mean, np.median, np.std, len],
                                     fill_value=np.nan)

/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/pivot.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return concat(pieces, keys=keys, axis=1)


The output DataFrame gives you a "hierarchical" column index — the three variables for each aggregate function. The row index is the `major` groups.

If you apply more index variables, the row indices will also become hierarchical! However, this can quickly make for a bloated DataFrame.

In [123]:
# print the header of the pivot table
nerdy_major_summary.head()

mean               \
variable                                           anxious bookish calm   
 None yet                                              7.0     3.0  3.0   
+ACI-+ACIAIg-hotel and restaurant management+AC...     2.0     2.0  7.0   
Aerospace Engineer                                     2.0     2.0  7.0   
Aerospace Engineering                                  3.0     4.0  3.0   
Agricultural Economics                                 2.0     2.0  6.0   

                                                    median               \
variable                                           anxious bookish calm   
 None yet                                              7.0     3.0  3.0   
+ACI-+ACIAIg-hotel and restaurant management+AC...     2.0     2.0  7.0   
Aerospace Engineer                                     2.0     2.0  7.0   
Aerospace Engineering                                  3.0     4.0  3.0   
Agricultural Economics                                 2.0     2.0  6.0   

                                                       std               \
variable                                           anxious bookish calm   
 None yet                                              NaN     NaN  NaN   
+ACI-+ACIAIg-hotel and restaurant management+AC...     NaN     NaN  NaN   
Aerospace Engineer                                     NaN     NaN  NaN   
Aerospace Engineering                                  NaN     NaN  NaN   
Agricultural Economics                                 NaN     NaN  NaN   

                                                       len               
variable                                           anxious bookish calm  
 None yet                                                1       1    1  
+ACI-+ACIAIg-hotel and restaurant management+AC...       1       1    1  
Aerospace Engineer                                       1       1    1  
Aerospace Engineering                                    1       1    1  
Agricultural Economics                                   1       1    1

<a id='examining_multiindex'></a>

### The Inner Workings of the MultiIndex

--- 

The `.names` attribute on the index and columns will show you the hierarchy of labels. The row index is `'major'`, and the two column indices are `None` and `'variable'` (The aggregate functions get no label from `.pivot_table()` in this case). 

If you print out the columns, you can see the data set has become a `pandas` `MultiIndex` object that has levels, labels, and names.

In [124]:
print(nerdy_major_summary.index.names)
print(nerdy_major_summary.columns.names)
print(nerdy_major_summary.columns)

[None]
[None, 'variable']
MultiIndex(levels=[['mean', 'median', 'std', 'len'], ['anxious', 'bookish', 'calm']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[None, 'variable'])


Indexing along the hierarchical column headers SHOULD NOT be done with chained bracket keys — i.e., including the top-level column label in the first bracket, and so on, down to the bottom level.

Instead you should use a tuple like index

In [125]:
nerdy_major_summary.loc[:,('mean','anxious')].head(2)

 None yet                                                    7.0
+ACI-+ACIAIg-hotel and restaurant management+ACIAIg-+ACI-    2.0
Name: (mean, anxious), dtype: float64

In [126]:
nerdy_major_summary.loc[:,('mean',['anxious','bookish'])].head(2)

mean        
variable                                           anxious bookish
 None yet                                              7.0     3.0
+ACI-+ACIAIg-hotel and restaurant management+AC...     2.0     2.0

<a id='multiindex_to_flat'></a>

### Getting Rid of the MultiIndex: "Flattening" Data

---

MultiIndex DataFrames hold great potential and are a cool concept. That being said, the overhead and confusion on how to subset/mask them is most often not worth it, especially when your data needs to be formatted for insertion into a model.

The most reliable way to "flatten" a MultiIndexed DataFrame is with the `.to_records()` function. To make this a new DataFrame, it needs to be wrapped in a `pd.DataFrame()` like so:

In [128]:
type(nerdy_major_summary.to_records())
nerdy_major_flat = pd.DataFrame(nerdy_major_summary.to_records())
nerdy_major_flat.head(2)

,index,"('mean', 'anxious')","('mean', 'bookish')","('mean', 'calm')","('median', 'anxious')","('median', 'bookish')","('median', 'calm')","('std', 'anxious')","('std', 'bookish')","('std', 'calm')","('len', 'anxious')","('len', 'bookish')","('len', 'calm')"
0,None yet,7.0,3.0,3.0,7.0,3.0,3.0,NaN,NaN,NaN,1,1,1
1,+ACI-+ACIAIg-hotel and restaurant management+A...,2.0,2.0,7.0,2.0,2.0,7.0,NaN,NaN,NaN,1,1,1


You can see that the new column names are tuples of the hierarchy of MultiIndexed columns. For example, you could convert these to new, more easily indexed columns with something like a list comprehension.

The **`.eval()`** function takes a string and trys to evaluate it as if it were a Python command.

**Use a list comprehension and the `.eval()` function to convert the flattened MultiIndexed columns to something more readable.**

In [ ]:
# replace the column names with list comprehension and eval


<a id='merging_joining_preface'></a>

### A Preface (independent practice): Merging and Joining With Long and Wide Format Data

---

You will be merging and joining data sets extensively throughout this course and in your future careers. However, it is important to note the differences between merging long and wide data sets together.

**Load in the data used above, but now split it so that the demographic variables are in one data set and the survey question answers are in another.** 

These data sets are in a wide format, and they both contain `subject_id`s to identify the questions' categories. 

As you may recall, the demographic responses have fewer observations.

In [129]:
n_demos_file = './datasets/NPAS_parsed_trunc_demo_sample.csv'
n_survey_file = './datasets/NPAS_parsed_trunc_survey.csv'
# load the files
demos_subset = pd.read_csv(n_demos_file)
survey = pd.read_csv(n_survey_file)

In [132]:
# print the header of the demos and survey
demos_subset.head()

,education,urban,gender,engnat,age,hand,religion,voted,married,familysize,major,subject_id
0,4.0,2.0,2.0,1.0,50.0,1.0,1.0,1.0,1.0,3.0,biophysics,1
1,3.0,1.0,2.0,2.0,22.0,1.0,1.0,1.0,1.0,2.0,biology,2
2,2.0,3.0,1.0,1.0,18.0,1.0,1.0,2.0,1.0,3.0,Geology,5
3,2.0,2.0,1.0,1.0,18.0,2.0,1.0,2.0,1.0,2.0,NaN,6
4,2.0,2.0,2.0,1.0,21.0,1.0,12.0,1.0,1.0,3.0,NaN,7


<a id='pandas_merge'></a>

### Use  `pandas`' `.merge()` function: Joining Long Format vs. Wide Format Data

---

As we have seen yesterday, the `.merge()` function comes built into a DataFrame. The first argument is another DataFrame you want to merge it with, and the `on` keyword argument is the key(s) by which you want the DataFrames to be "matched."

We are specifying `how='inner'` here, which means that the key must be present in both DataFrames to have the corresponding rows included in the output. Because the demographics data set has fewer `subject_id`s, it will only merge the `subject_id` rows from the survey data set that are also present in the demographics data set.

**Combine the survey and demographic wide format data sets using `.merge()`.**

In [135]:
demos_survey = demos_subset.merge(survey, on='subject_id',how='inner')
demos_survey.shape

(700, 57)

In [136]:
# print the merged data header
demos_survey.head()

,education,urban,gender,engnat,age,hand,religion,voted,married,familysize,major,subject_id,race_white,race_nerdy,race_native_american,writing_novel,read_tech_reports,online_over_inperson,introspective,hobbies_over_people,books_over_parties,bookish,libraries_over_publicspace,race_native_austrailian,like_hard_material,race_hispanic,diagnosed_autistic,play_many_videogames,race_arab,race_asian,interested_science,playes_rpgs,in_advanced_classes,collect_books,intelligence_over_appearance,watch_science_shows,academic_over_social,like_science_fiction,like_dry_topics,race_black,calm,disorganized,extraverted,dependable,critical,opennness,anxious,sympathetic,reserved,conventional,was_odd_child,prefer_fictional_people,enjoy_learning,excited_about_research,strange_person,like_superheroes,socially_awkward
0,4.0,2.0,2.0,1.0,50.0,1.0,1.0,1.0,1.0,3.0,biophysics,1,1.0,0.0,0.0,1.0,4.0,3.0,3.0,1.0,4.0,4.0,5.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,4.0,4.0,4.0,5.0,3.0,5.0,2.0,4.0,1.0,0.0,6.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,1.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0
1,3.0,1.0,2.0,2.0,22.0,1.0,1.0,1.0,1.0,2.0,biology,2,1.0,0.0,0.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,0.0,4.0,0.0,2.0,5.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,2.0,6.0,1.0,3.0,6.0,6.0,7.0,2.0,7.0,1.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0
2,2.0,3.0,1.0,1.0,18.0,1.0,1.0,2.0,1.0,3.0,Geology,5,1.0,0.0,0.0,3.0,5.0,3.0,4.0,4.0,4.0,3.0,4.0,0.0,4.0,0.0,2.0,5.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,3.0,0.0,4.0,3.0,2.0,5.0,3.0,5.0,5.0,4.0,5.0,4.0,4.0,4.0,2.0,5.0,5.0,3.0,4.0
3,2.0,2.0,1.0,1.0,18.0,2.0,1.0,2.0,1.0,2.0,NaN,6,0.0,0.0,0.0,1.0,4.0,3.0,1.0,3.0,5.0,4.0,3.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,5.0,4.0,3.0,5.0,3.0,4.0,4.0,5.0,5.0,1.0,6.0,5.0,3.0,2.0,1.0,7.0,1.0,5.0,5.0,1.0,1.0,5.0,1.0,4.0,5.0,3.0,5.0
4,2.0,2.0,2.0,1.0,21.0,1.0,12.0,1.0,1.0,3.0,NaN,7,1.0,0.0,0.0,3.0,5.0,4.0,5.0,4.0,5.0,3.0,4.0,0.0,4.0,0.0,2.0,3.0,0.0,0.0,5.0,4.0,3.0,5.0,5.0,3.0,3.0,3.0,5.0,0.0,1.0,7.0,3.0,5.0,6.0,6.0,7.0,6.0,5.0,4.0,3.0,4.0,4.0,3.0,5.0,2.0,5.0


**Convert the demographic and survey data into long format using `.melt()`.**

- For the demographic DataFrame, specify two `id_vars` — `gender` and `subject_id`.
- For the survey DataFrame, only specify `subject_id` for `id_vars`.

In [144]:
# melt the demographic data
demo_long=pd.melt(demos_subset,id_vars=['subject_id','gender'])
demo_long.head()

,subject_id,gender,variable,value
0,1,2.0,education,4
1,2,2.0,education,3
2,5,1.0,education,2
3,6,1.0,education,2
4,7,2.0,education,2


In [145]:
# melt the survey data
survey_long=survey_long=pd.melt(survey,id_vars=['subject_id'])
survey_long.head()

,subject_id,variable,value
0,0,race_white,1.0
1,1,race_white,1.0
2,2,race_white,1.0
3,3,race_white,1.0
4,4,race_white,1.0


**Merge the long form data sets together, just like we did previously with the wide format data.**

Here, we will still merge on `subject_id`, using `'inner'` for the `how` variable. We have duplicate named columns in each of these DataFrames (`variable` and `value`). We can specify `suffixes=('_survey','_demo')` to give the instances of the survey and demographic DataFrames appropriate column names when they are joined together.

In [146]:
# merge the survey and demo data
demo_long.merge(survey_long,on='subject_id',how='inner',suffixes=['_survey','_demo'])

,subject_id,gender,variable_survey,value_survey,variable_demo,value_demo
0,1,2.0,education,4,race_white,1.0
1,1,2.0,education,4,race_nerdy,0.0
2,1,2.0,education,4,race_native_american,0.0
3,1,2.0,education,4,writing_novel,1.0
4,1,2.0,education,4,read_tech_reports,4.0
5,1,2.0,education,4,online_over_inperson,3.0
6,1,2.0,education,4,introspective,3.0
7,1,2.0,education,4,hobbies_over_people,1.0
8,1,2.0,education,4,books_over_parties,4.0
9,1,2.0,education,4,bookish,4.0
